In [1]:
# 计算每个数字的贡献值
# 比如[1,2,4,5,3,6,2]
# 对于3 他可以作为[4,5,3,6],[5,3,6],[3,6],[4,5,3],[5,3],[3]
from typing import List


class Solution:

    def sumSubarrayMins(self, arr: List[int]) -> int:
        tmp = 10**9 + 7
        leftMin, rightMin = [-1] * len(arr), [len(arr)] * len(arr)  # 左边<cur的值
        stack = []
        for i, cur in enumerate(arr):
            while stack and arr[stack[-1]] > cur:
                stack.pop()
            if stack:
                leftMin[i] = stack[-1]
            stack.append(i)
        stack = []
        for i in range(len(arr) - 1, -1, -1):
            cur = arr[i]
            while stack and arr[stack[-1]] >= cur:
                stack.pop()
            if stack:
                rightMin[i] = stack[-1]
            stack.append(i)
        ans = 0
        for i, cur in enumerate(arr):
            ans += (i - leftMin[i]) * (rightMin[i] - i) * cur
            ans %= tmp
        return ans


In [9]:
# 线段树解法 也是基于贡献值法
class Node:

    def __init__(self, si, ei):
        self.si = si
        self.ei = ei
        self.mi = si + ei >> 1
        self.left = None
        self.right = None
        self.val = float('inf'), None  # 区间最小值及索引
        self.lazy = None  # 区间已更新但未下发 (None:已经下发)


class SegmentTree:

    def __init__(self, si, ei):
        self.root = Node(si, ei)

    def pushdown(self, node: Node):
        if not node.left: node.left = Node(node.si, node.mi)
        if not node.right: node.right = Node(node.mi + 1, node.ei)
        if node.lazy is None:
            return
        node.left.val = node.lazy[0], node.left.si
        node.right.val = node.lazy[0], node.right.si
        node.left.lazy = node.lazy[0], node.left.si
        node.right.lazy = node.lazy[0], node.right.si
        node.lazy = None

    def update(self, node: Node, l, r, val):  # [l,r]设为val node初始为root再逐步减半
        if node.ei < l or node.si > r:
            return
        if l <= node.si and node.ei <= r:
            node.val = val, node.si
            node.lazy = val, node.si
            return
        self.pushdown(node)
        if node.left.ei >= l:
            self.update(node.left, l, r, val)
        if node.right.si <= r:
            self.update(node.right, l, r, val)
        node.val = min(node.left.val, node.right.val, key=lambda x: x[0])

    def query(self, node: Node, l, r):  # 查询[l,r]最小值 node初始为root再逐步减半
        if node.ei < l or node.si > r:
            return float('inf'), None
        if l <= node.si and node.ei <= r:
            return node.val
        self.pushdown(node)
        ans = float('inf'), None
        if node.left.ei >= l:
            ans = min(ans, self.query(node.left, l, r), key=lambda x: x[0])
        if node.right.si <= r:
            ans = min(ans, self.query(node.right, l, r), key=lambda x: x[0])
        return ans


from typing import List


class Solution:

    def sumSubarrayMins(self, arr: List[int]) -> int:
        Mod = 10**9 + 7
        st = SegmentTree(0, len(arr) - 1)
        for i, n in enumerate(arr):
            st.update(st.root, i, i, n)

        def dfs(si, ei):  # 对[si,ei]分治
            if si > ei:
                return 0
            if si == ei:
                return arr[si]
            m, i = st.query(st.root, si, ei)
            cnt = 0
            cnt += m * (i - si + 1) * (ei - i + 1) % Mod
            cnt += (dfs(si, i - 1) + dfs(i + 1, ei)) % Mod
            return cnt

        return dfs(0, len(arr) - 1)


s = Solution()
s.sumSubarrayMins([11, 81, 94, 43, 3])


444